# 📘  프롬프트

---

## 01. PromptTemplate, 부분변수(partial_variables)

### 📌 개념
- **PromptTemplate** : 입력값을 템플릿에 대입해서 프롬프트를 생성  
- **partial_variables** : 일부 변수를 미리 고정시켜 두고 나머지만 실행 시점에 채움  



In [1]:
### 💻 예시 코드
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("오늘 {subject}에 대해 {action} 해줘.")

# 일반 사용
print(template.format(subject="LangChain", action="설명"))

# partial_variables 사용
partial_template = PromptTemplate.from_template(
    "오늘 {subject}에 대해 {action} 해줘.",
    partial_variables={"action": "요약"}
)
print(partial_template.format(subject="프롬프트 엔지니어링"))

오늘 LangChain에 대해 설명 해줘.
오늘 프롬프트 엔지니어링에 대해 요약 해줘.


## 02. yaml 파일로부터 프롬프트 템플릿 로드(load_prompt) 
### 📌 개념

- YAML 파일에 프롬프트 템플릿 정의 후 불러오기 가능

- 협업 시 프롬프트 관리 및 버전 관리에 유용

In [4]:
# from langchain.prompts import load_prompt

# """
# _type: prompt
# input_variables:
#   ["product"]
# template: "Write a creative ad for the following product: {product}."
# """

# prompt = load_prompt("prompt.yaml")
# print(prompt.format(product="스마트워치"))

## 03. ChatPromptTemplate
### 📌 개념

- 대화형 프롬프트를 구성할 때 사용

- SystemMessage, HumanMessage, AIMessage 등을 조합

In [5]:
from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 유용한 번역 도우미입니다."),
    ("human", "{text}을 한국어로 번역해줘.")
])

print(chat_prompt.format_messages(text="Good morning"))

[SystemMessage(content='당신은 유용한 번역 도우미입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Good morning을 한국어로 번역해줘.', additional_kwargs={}, response_metadata={})]


## 04. MessagesPlaceHolder 
### 📌 개념

- 대화 히스토리를 그대로 프롬프트에 삽입할 때 사용

- ChatPromptTemplate과 함께 쓰임

In [6]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 친절한 어시스턴트야."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

print(prompt.format_messages(
    history=[("human", "안녕!"), ("ai", "안녕하세요 :)")],
    input="오늘 날씨 어때?"
))

[SystemMessage(content='너는 친절한 어시스턴트야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕!', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요 :)', additional_kwargs={}, response_metadata={}), HumanMessage(content='오늘 날씨 어때?', additional_kwargs={}, response_metadata={})]


## 05. 퓨샷프롬프트(FewShotPromptTemplate) 
### 📌 개념

- Few-shot learning : 프롬프트에 예시를 함께 제공해 성능 향상

- 여러 input/output 예시를 넣어서 모델 가이딩

In [7]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

examples = [
    {"word": "happy", "synonym": "joyful"},
    {"word": "sad", "synonym": "unhappy"}
]

example_prompt = PromptTemplate(
    input_variables=["word", "synonym"],
    template="단어: {word}\n동의어: {synonym}\n"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="단어: {word}\n동의어:",
    input_variables=["word"]
)

print(few_shot_prompt.format(word="fast"))

단어: happy
동의어: joyful


단어: sad
동의어: unhappy


단어: fast
동의어:


## 06. 예제 선택기(Example Selector) 
### 📌 개념

- 제공된 예시 중 일부만 선택해서 few-shot에 삽입

- 데이터셋이 클 때 유용

In [8]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {"word": "happy", "synonym": "joyful"},
    {"word": "sad", "synonym": "unhappy"},
    {"word": "angry", "synonym": "mad"},
]

example_prompt = PromptTemplate(
    input_variables=["word", "synonym"],
    template="단어: {word}\n동의어: {synonym}\n"
)

selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25
)

print(selector.select_examples({"word": "excited"}))

[{'word': 'happy', 'synonym': 'joyful'}, {'word': 'sad', 'synonym': 'unhappy'}, {'word': 'angry', 'synonym': 'mad'}]


## 07. MaxMarginalRelevance(MMR) 알고리즘 
### 📌 개념

- 예제 선택 시 다양성과 관련성을 동시에 고려

- 비슷한 예시만 선택되는 것을 방지

In [10]:
# from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector

# selector = MaxMarginalRelevanceExampleSelector.from_examples(
#     examples,
#     example_prompt,
#     vectorizer=lambda x: [ord(c) for c in x["word"]],  # 단순 벡터화 예시
#     k=2
# )

# print(selector.select_examples({"word": "joy"}))

## 08. FewShotChatMessagePromptTemplate 
### 📌 개념

- ChatPromptTemplate + FewShotPromptTemplate 결합

- 대화 맥락에서도 few-shot 제공 가능

In [12]:
# from langchain.prompts import FewShotChatMessagePromptTemplate

# examples = [
#     {"input": "2+2", "output": "4"},
#     {"input": "3*3", "output": "9"}
# ]

# example_prompt = ChatPromptTemplate.from_messages([
#     ("human", "{input}"),
#     ("ai", "{output}")
# ])

# few_shot_chat_prompt = FewShotChatMessagePromptTemplate(
#     examples=examples,
#     example_prompt=example_prompt,
#     suffix=[("human", "{input}")],
#     input_variables=["input"]
# )

# print(few_shot_chat_prompt.format_messages(input="5-2"))

## 09. 목적에 맞는 예제 선택기(CustomExampleSelector) 
### 📌 개념

- ExampleSelector를 상속받아 원하는 규칙 기반 선택 가능

In [14]:
# from langchain.prompts.example_selector.base import BaseExampleSelector

# class CustomSelector(BaseExampleSelector):
#     def __init__(self, examples):
#         self.examples = examples
    
#     def select_examples(self, input_variables):
#         # 단순히 "첫 번째 예시만" 반환
#         return [self.examples[0]]

# examples = [
#     {"word": "happy", "synonym": "joyful"},
#     {"word": "sad", "synonym": "unhappy"}
# ]

# selector = CustomSelector(examples)
# print(selector.select_examples({"word": "fast"}))

## 10. 프롬프트 허브(LangChain Hub)
### 📌 개념

- LangChain에서 제공하는 공식 프롬프트 저장소

- 미리 정의된 고품질 프롬프트 불러와 사용 가능

In [15]:
from langchain import hub

# 불러오기 (예: "hwchase17/react")
prompt = hub.pull("hwchase17/react")

print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


## ✅ 정리

- PromptTemplate : 기본 프롬프트 템플릿

- partial_variables : 변수를 일부 고정

- ChatPromptTemplate, MessagesPlaceholder : 대화형 프롬프트

- FewShotPromptTemplate, Example Selector : few-shot 학습 지원

- MMR : 다양성과 관련성 동시 고려

- LangChain Hub : 공개 프롬프트 활용